In [3]:
import dask.dataframe as dd
import numpy as np

input_file = "/mnt/c/Users/hoang/FileCSV_DACN_2025/iot23_cleaned.csv"

dictTypes = {}

df = dd.read_csv(input_file)
# dtype={'detailed-label': 'object'}, assume_missing=True

# print(df.columns)
for col in df.columns:
    if col.startswith('proto') == True:
        dictTypes[col] = 'int32'
    elif col.startswith('service_') == True:
        dictTypes[col] = 'int32'
    elif col.startswith('detailed-label'):
        dictTypes[col] = 'str'
    else:
        dictTypes[col]='float32'
del df

df = dd.read_csv(input_file, dtype = dictTypes)
df = df.drop(columns=['service_0', 'label'])
# phân loại đa nhãn, k dùng
df = df.replace(r'[N|n][a|A][N|n]', 0)
df = df.replace(np.nan, 0)
print(df.dtypes)
print(df.tail())

id.orig_h                 float32
id.orig_p                 float32
id.resp_h                 float32
id.resp_p                 float32
duration                  float32
orig_bytes                float32
resp_bytes                float32
conn_state                float32
missed_bytes              float32
history                   float32
orig_pkts                 float32
orig_ip_bytes             float32
resp_pkts                 float32
resp_ip_bytes             float32
detailed-label    string[pyarrow]
proto_icmp                  int32
proto_tcp                   int32
proto_udp                   int32
service_dhcp                int32
service_dns                 int32
service_http                int32
service_irc                 int32
service_ssh                 int32
service_ssl                 int32
dtype: object
        id.orig_h  id.orig_p  id.resp_h  id.resp_p  duration  orig_bytes  \
536603   3.232261   0.422713   2.908634   0.350872       0.0         0.0   
536604   3.232261 

In [14]:
label_counts = df['detailed-label'].value_counts().compute()

In [15]:
print(label_counts)

detailed-label
Attack                                   9398
Okiru-Attack                                3
C&C-HeartBeat-Attack                      834
PartOfAHorizontalPortScan-Attack            5
FileDownload                               18
C&C-Mirai                                   2
DDoS                                 16108989
C&C-FileDownload                           53
Okiru                                60938340
C&C-HeartBeat                           31528
C&C-HeartBeat-FileDownload                 11
C&C                                     17657
PartOfAHorizontalPortScan           213583302
C&C-Torii                                  23
0                                    30771095
C&C-PartOfAHorizontalPortScan             797
Name: count, dtype: int64[pyarrow]


In [ ]:

import matplotlib.pyplot as plt
ordered_labels = [0, 1, 2, 3]
ordered_counts = [label_counts.get(label, 0) for label in ordered_labels] 
print(label_counts)
labels = ["Benign","DDoS","DoS","Mirai"]
# Vẽ biểu đồ cột
plt.figure(figsize=(9, 5))
plt.bar(labels, ordered_counts, color='skyblue', edgecolor='black')
plt.xlabel("Nhãn (Classes)")
plt.ylabel("Số lượng mẫu (Frequency)")
plt.title("Tỷ lệ nhãn trong Dataset")
plt.xticks(range(len(labels)) ,labels, rotation =45)
plt.show()

In [4]:
df = df[df['detailed-label'].isin(['Okiru', 'PartOfAHorizontalPortScan', '0'])]

label_map = {
    '0': 0,
    'PartOfAHorizontalPortScan': 1,
    'Okiru': 2
}

df['detailed-label'] = df['detailed-label'].map(label_map).astype('int32')

/home/hoangvn/miniconda3/miniconda3/envs/doan/lib/python3.9/site-packages/dask_expr/_collection.py:4190: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('detailed-label', 'float64'))

  warnings.warn(meta_warning(meta))


In [5]:
print(df.tail(100))

        id.orig_h  id.orig_p  id.resp_h  id.resp_p  duration  orig_bytes  \
536508   3.232261   0.532167   2.468357   0.350872       0.0         0.0   
536509   3.232261   0.677962   2.468413   0.350872       0.0         0.0   
536510   3.232261   0.650508   2.468396   0.350872       0.0         0.0   
536511   3.232261   0.105631   2.468380   0.350872       0.0         0.0   
536512   3.232261   0.071794   2.468401   0.350872       0.0         0.0   
...           ...        ...        ...        ...       ...         ...   
536603   3.232261   0.422713   2.908634   0.350872       0.0         0.0   
536604   3.232261   0.103287   1.170498   0.350872       0.0         0.0   
536605   3.232261   0.154855   3.229167   0.350872       0.0         0.0   
536606   2.825261   0.313650   3.232261   0.025953       0.0         0.0   
536607   3.227416   0.313650   3.232261   0.025953       0.0         0.0   

        resp_bytes  conn_state  missed_bytes   history  ...  detailed-label  \
536508  

# Generator + Train

In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from datetime import datetime, timedelta

#Global var 
batch_size = 1024
ratio_test_all = 0.2

from dask_ml.model_selection import train_test_split 
# Bước 1: Tách 80% train, 20% còn lại (val + test)
train_df, val_test_df = train_test_split(df, test_size=0.20, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.75, random_state=42)


features_len = len(df.columns())-1
def dask_to_tf_dataset(dask_df, batch_size, num_classes): 
    def generator():
        for batch in dask_df.to_delayed():
            batch=batch.compute()  
            if batch.empty:
                continue

            X = batch.drop(columns='detailed-label').values.astype(np.float32)
            y = batch['detailed-label'].values
            # y_onehot = to_categorical(y, num_classes=num_classes)  

            num_splits = max(1, len(X) // batch_size)  # Đảm bảo không chia nhỏ quá mức
            X_batches = np.array_split(X, num_splits)
            y_batches = np.array_split(y, num_splits)

            for X_batch, y_batch in zip(X_batches, y_batches):
                yield X_batch, y_batch
                
    output_signature = ( 
        tf.TensorSpec(shape=(None, features_len), dtype=tf.float32), 
        tf.TensorSpec(shape=(None, 3), dtype=tf.int32),
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

In [ ]:
batchSize = 1024
train_gen = dask_to_tf_dataset(train_df, 1024).repeat()
val_gen = dask_to_tf_dataset(val_df, 1024).repeat()
test_gen = dask_to_tf_dataset(test_df, 1024)

In [ ]:
import math
n_samples = df.shape[0].compute()
steps_per_epoch = math.ceil(n_samples / batch_size)
validation_steps = math.ceil(steps_per_epoch / 40)
print("Steps/Epoch: ", steps_per_epoch)

# shape
features, labels = next(iter(train_gen))
input_shape = (features.shape[1], 1)
output_shape = labels.shape[1]

print(f"Input Shape: {input_shape}")

from tensorflow import keras
# Định nghĩa mô hình CNN
# VGG, ...
# Conv2D, tabular, ...
# HE, tính tương thích của HE với CNN
# Tính chất data in, out; Học tăng cường
start_time = datetime.now()

model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv1D(filters=128, kernel_size=7, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv1D(filters=128, kernel_size=7,  padding="same",activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(output_shape, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# for batch in dataloader:
#     X_batch = batch[:, :-1]
#     y_batch = batch[:, -1]
#     y_onehot = to_categorical(y_batch, num_classes=10)
    
#     model.train_on_batch(X_batch, y_onehot, verbose=1)
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger("Centralized_Log/"+ datetime.now().strftime("%Hh%Mp__%d-%m-%Y")+".csv" , append=True)
model.fit(train_gen, epochs=25,  validation_data=val_gen, 
          validation_steps=validation_steps, steps_per_epoch=steps_per_epoch, verbose = 1, callbacks=[csv_logger])


end_time = datetime.now()
simulated_time = end_time - start_time

# Lưu mô hình
model.save("Centralized_Model/cnn_model_2-0_batch512_" + datetime.now().strftime("%Hh%Mp__%d-%m-%Y")+".keras")

print(f"Simulated time: {simulated_time}")

2025-05-04 05:30:44.858857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746336644.991220    1312 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746336645.029427    1312 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746336645.358698    1312 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746336645.358786    1312 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746336645.358787    1312 computation_placer.cc:177] computation placer alr

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is using GPU:  True
